In [8]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from pathlib import Path
import os
import string
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
import yaml

# import ruamel.yaml as ryaml
import json
# import ast

os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Raj-Narayanan-B'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '8af4cc66be8aec751397fd525e47ae395fa67442'

# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier, 
                              HistGradientBoostingClassifier, StackingClassifier, VotingClassifier)
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import (balanced_accuracy_score, f1_score,
                             accuracy_score, confusion_matrix)

from sklearn.model_selection import train_test_split, StratifiedKFold

from src.constants import *
from src.utils import load_yaml, parameter_tuning, stage_2_processing_function,eval_metrics, save_yaml, load_binary
# from src.config.configuration_manager import ConfigurationManager
# from src.entity.entity_config import DataSplitConf
# from src.components.stage_5_model_tuning_and_tracking
os.chdir('f:\\iNeuron\\Projects\\Scania_Truck_Failures')

In [90]:
df = pd.read_csv("F:\iNeuron\Projects\scania_failures_2\\artifacts\data\processed\stage_1_processing\preprocessed_train_data.csv").iloc[:10000:]
print("Shape: ",df.shape)
print("NA: ", df.isna().sum().unique())
print("Target Value_Counts: ", df['class'].value_counts())
# os.remove("F:\iNeuron\Projects\scania_failures_2\\artifacts\preprocessor\preprocessor.joblib")
# train,test = train_test_split(df, test_size = 0.25, random_state=8)
transformed_train_df = pd.read_csv("F:\iNeuron\Projects\scania_failures_2\\artifacts\data\processed\stage_2_processing\processed_train_data.csv")
transformed_test_df = pd.read_csv("F:\iNeuron\Projects\scania_failures_2\\artifacts\data\processed\stage_2_processing\processed_test_data.csv")
print(f"\nTransformed_train_df.shape: {transformed_train_df.shape},\nTransformed_test_df.shape: {transformed_test_df.shape}")
x_train, y_train = transformed_train_df.drop(columns='class'), transformed_train_df['class']
x_test, y_test = transformed_test_df.drop(columns = 'class'), transformed_test_df['class']
print(f"\nx_train.shape: {x_train.shape},\ny_train.shape: {y_train.shape},\nx_test.shape: {x_test.shape},\ny_test.shape: {y_test.shape}")
print(f"\nNA in train_data after preprocessing: {x_train.isna().sum().unique()}")
print(f"NA in test_data after preprocessing: {x_test.isna().sum().unique()}")
print(f"\nValue_counts in train_data after preprocessing: {y_train.value_counts()}")
print(f"Value_counts in test_data after preprocessing: {y_test.value_counts()}")

Shape:  (10000, 171)
NA:  [   0 7782  595 2475  443  124  116  107  790  115   99  489  125  127
 3729 4459 6538 7287 7695 7938 8086 8171  128   39  126  563   81  785
   65 1601 1658 2309  718 1719]
Target Value_Counts:  class
0    9805
1     195
Name: count, dtype: int64

Transformed_train_df.shape: (88330, 171),
Transformed_test_df.shape: (29406, 171)

x_train.shape: (88330, 170),
y_train.shape: (88330,),
x_test.shape: (29406, 170),
y_test.shape: (29406,)

NA in train_data after preprocessing: [0]
NA in test_data after preprocessing: [0]

Value_counts in train_data after preprocessing: class
0    44165
1    44165
Name: count, dtype: int64
Value_counts in test_data after preprocessing: class
0    14703
1    14703
Name: count, dtype: int64


In [ ]:
from sklearn.datasets import make_classification

In [ ]:
t_x, t_y = make_classification(n_samples=1000,
                               n_features=170)
t_x_df =pd.DataFrame(t_x, columns = x_test.columns)
t_x_df.head()

In [ ]:
t_x_df['class'] = t_y
t_x_df.head()

In [ ]:
source = mlflow.search_registered_models(filter_string = "tags.model_type ilike 'champion'")[0].latest_versions[0].source

In [ ]:
model = mlflow.sklearn.load_model(source)

In [ ]:
model

In [ ]:
t_y_pred = model.predict(t_x_df.drop(columns=['class']))

In [ ]:
eval_metrics(y_true=t_x_df['class'], y_pred = t_y_pred)

In [ ]:
t_x_df.describe()

In [ ]:
import random
input_ = np.array([random.random() for i in range(170)]).reshape(1,-1)

In [ ]:
model.predict(input_)

In [ ]:
!pip list

In [ ]:
from src.utils import load_yaml
import pandas as pd

sample_df = pd.DataFrame(load_yaml(r"F:\iNeuron\Projects\scania_failures_2\artifacts\metrics\best_metric.yaml").VotingClassifier).T.sort_values(by = 'Accuracy_Score', ascending=False)

In [ ]:
mlflow.search_registered_models(filter_string="tags.model_type ilike 'Challenger'AND tags.model_type ilike 'Champion'")

In [ ]:
sources = {}
models = {}
for i in range(1):
    model_name = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[i].name
    sources[model_name] = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[i].latest_versions[0].source
    model = mlflow.sklearn.load_model(sources[model_name])
#     model_name = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Challenger'")[i].name
    models[model_name] = model
    # print(type(model_name))
    # print(model_name)
models

In [ ]:
champion_source = {}
for i in range(1):
    model_name = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[i].name
    champion_source[model_name] = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[i].latest_versions[0].source + "/model.pkl"

In [ ]:
champion_source

In [ ]:
for i in range(2):
    model_name = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Challenger'")[i].name
    champion_source[model_name] = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Challenger'")[i].latest_versions[0].source + "/model.pkl"
champion_source

In [ ]:
champion_source

In [ ]:
from src.utils import save_yaml


save_yaml(file = champion_source,filepath='source.yaml')

In [ ]:
champion_source

In [ ]:
mlflow-artifacts:/cd6eb5acbadd414fb5d32e02d983d99b/b2aea20ef6094e1a94b1567aea90753f/artifacts/candidate_Voting_Classifier/model.pkl

In [ ]:
sources

In [ ]:
sample_df['models'] = models

In [ ]:
sample_df

In [ ]:
sample_df.iloc[:1, :]['Accuracy_Score'].index[0]

In [ ]:
sample_df.iloc[:1, :]['Accuracy_Score'].values[0]

In [ ]:
sample_df.iloc[:1,:]['models'].values[0]

In [ ]:
for key,value in sources.items():
    sources[key] = value+"/model.pkl"

In [ ]:
sources

In [ ]:
from src.utils import save_yaml

source = {
    'candidate_Voting_Classifier': 'mlflow-artifacts:/cd6eb5acbadd414fb5d32e02d983d99b/b2aea20ef6094e1a94b1567aea90753f/artifacts/candidate_Voting_Classifier',
    'challenger_LGBMClassifier' : 'mlflow-artifacts:/f2a1d0cb47994e43887a7bf8ff11fbc0/4f79ae8649044b24a593ec1c748e012b/artifacts/challenger_LGBMClassifier',
    'candidate_Stacked_Classifier' : 'mlflow-artifacts:/aacd27cc7a9d429ab2439e37a4e8cdfb/d3eb9ead8a5c49f9b8299efea62d58a2/artifacts/candidate_Stacked_Classifier'
}
source

In [ ]:
save_yaml(file = champion_source, filepath= r"F:\iNeuron\Projects\scania_failures_2\champion_source.yaml")

In [ ]:
from pprint import pprint

mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[0].latest_versions[0].run_id

In [ ]:
from pprint import pprint
import json
model_detail = mlflow.get_run('b2aea20ef6094e1a94b1567aea90753f').data.tags['mlflow.log-model.history']
pprint(json.loads(model_detail)[0], compact = True)

In [ ]:
mlflow-artifacts:/cd6eb5acbadd414fb5d32e02d983d99b/b2aea20ef6094e1a94b1567aea90753f/artifacts/candidate_Voting_Classifier/model.pkl

In [ ]:
mlflow-artifacts:/f2a1d0cb47994e43887a7bf8ff11fbc0/4f79ae8649044b24a593ec1c748e012b/artifacts/challenger_LGBMClassifier/model.pkl

In [ ]:
mlflow-artifacts:/aacd27cc7a9d429ab2439e37a4e8cdfb/d3eb9ead8a5c49f9b8299efea62d58a2/artifacts/candidate_Stacked_Classifier/model.pkl

In [ ]:
grad_boost_params = {"criterion": "squared_error",
        "loss": "exponential",
        'max_features': 'sqrt',
        'n_estimators': 103}

log_reg_params = {'penalty': 'l2'}

xgb_params = {'booster': 'gbtree',
        'learning_rate': 2.810313927552631,
        'n_estimators': 658,
        'tree_method': 'approx'}

grd_boost_model = GradientBoostingClassifier(**grad_boost_params)
log_reg_model = LogisticRegression(**log_reg_params)
xgb_model = XGBClassifier(**xgb_params)
estimators = [("grd_boost",grd_boost_model),
              ("log_reg",log_reg_model),
              ("xgb", xgb_model)]

final_estimator = XGBClassifier()
stacked_classifer = StackingClassifier(estimators=estimators,
                                       final_estimator=final_estimator,
                                       cv=5,
                                       passthrough = True)
voting_classifier = VotingClassifier(estimators = estimators,
                                                    voting = "hard",
                                                    weights = None,
                                                    n_jobs = -1,
                                                    verbose = True)

In [27]:
from dagshub import get_repo_bucket_client
s3 = get_repo_bucket_client("Raj-Narayanan-B/scania_truck")
s3.upload_file(
    Bucket="scania_truck",  # name of the repo
    Filename=r"F:\iNeuron\Projects\Data\Scania Truck Failures\aps_failure_training_set.csv",  # local path of file to upload
    Key="remote_aps_failure_training_set.csv",  # remote path where to upload the file
)

Client created. Use the name of the repo (scania_truck) as the name of the bucket

[2024-02-13 23:24:37,024: INFO: helpers: Client created. Use the name of the repo (scania_truck) as the name of the bucket]


In [31]:
s3.download_file(
    Bucket="scania_truck",  # name of the repo
    Key="remote_aps_failure_training_set.csv",  #  remote path from where to download the file
    Filename=r"F:\iNeuron\Projects\scania_failures_2\notebooks\sample_train.csv",  # local path where to download the file
)

In [28]:
s3.upload_file(
    Bucket="scania_truck",  # name of the repo
    Filename=r"F:\iNeuron\Projects\Data\Scania Truck Failures\aps_failure_test_set.csv",  # local path of file to upload
    Key="remote_aps_failure_testing_set.csv",  # remote path where to upload the file
)

In [30]:
s3.download_file(
    Bucket="scania_truck",  # name of the repo
    Key="remote_aps_failure_testing_set.csv",  #  remote path from where to download the file
    Filename=r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\sample.csv",  # local path where to download the file
)


In [ ]:
for i in range(len(estimators)):
    print(estimators[i][0])

In [ ]:
print([estimators[i][0] for i in range(len(estimators))])

In [ ]:
from yaml import safe_dump
yaml.dump(data = stacked_classifer.get_params(),
                  stream = open('F:\iNeuron\Projects\scania_failures_2\sample.yaml', 'w'),
                  indent = 4)

In [ ]:
with open('F:\iNeuron\Projects\scania_failures_2\sample.yaml') as yaml_file:
    yaml_file = yaml.load(yaml_file, Loader = yaml.CLoader)
yaml_file

In [ ]:
yaml_file['estimators'][1][1]

In [ ]:
b_estimators = dict(grd_boost = grd_boost_model,
                    log_reg = log_reg_model,
                    xgb =  xgb_model)
estimators_ = list(zip(b_estimators.keys(),b_estimators.values()))
for i in range(len(estimators_)):
    estimators_[0][i] = str(estimators_[0][i])

In [ ]:
b_estimators

In [ ]:
estimators_[0]

In [ ]:
keys, values = zip(*estimators_)

In [ ]:
keys

In [ ]:
sample = {}
for i in range(len(keys)):
    sample[keys[i]] = values[i]
sample

In [ ]:
stacked_classifer.fit(X = x_train, y = y_train)
y_pred_ = stacked_classifer.predict(X = x_test)
cost_ = eval_metrics(y_true = y_test, y_pred=y_pred_)
print(cost_)

In [ ]:
voting_classifier.fit(x_train, y_train)
y_pred = voting_classifier.predict(x_test)
cost = eval_metrics(y_true = y_test , y_pred = y_pred)
cost

In [ ]:
voting_classifier.get_params()['flatten_transform']

In [ ]:
StackingClassifier()

In [ ]:
key, value = zip(*stacked_classifer.get_params()['estimators'])

In [ ]:
report = {}
report['estimators'] = stacked_classifer.get_params()['estimators']
report['final_estimator'] = stacked_classifer.get_params()['final_estimator']
report


In [ ]:
report_1 = {}
report_1['stack_method'] = stacked_classifer.get_params()['stack_method']
report_1['passthrough'] = stacked_classifer.get_params()['passthrough']
report_1

In [ ]:
for i in report_1.keys():
    report[i] = report_1[i]

In [ ]:
report.keys()

In [ ]:
sc = StackingClassifier(**report)
sc


In [ ]:
sc.get_params()

In [ ]:
key

In [ ]:
value

In [ ]:
{'Stacked_Classifier': {'Logistic_Regression': {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 
'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}, 'SGD_Classifier': {'alpha': 0.0001, 'average': False, 'class_weight': None, 'early_stopping': False, 'epsilon': 0.1, 'eta0': 0.0, 'fit_intercept': True, 'l1_ratio': 
0.15, 'learning_rate': 'optimal', 'loss': 'huber', 'max_iter': 1000, 'n_iter_no_change': 5, 'n_jobs': None, 'penalty': 'l2', 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'tol': 0.001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}, 'Decision_Tree_Classifier': {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'log_loss', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}, 'KNN_Classifier': {'algorithm': 'brute', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 11, 'p': 2, 'weights': 'uniform'}}}

In [ ]:
yaml_r = yaml.YAML(typ='rt')

# Read data from the YAML file
with open('params.yaml') as yaml_file:
    params_config = yaml_r.load(yaml_file)

model_name = "Stacked_Classifier"
model_class = stacked_classifer

In [ ]:
params_config

In [ ]:
# criterion='squared_error', 
# loss='exponential',
# max_features='sqrt', 
# n_estimators=103

grad_boost_params

In [ ]:
model.get_params()['grd_boost'].get_params()

In [ ]:
model.get_params()['passthrough']

In [ ]:
model.get_params()

In [ ]:
import optuna
def optuna_objective(trial):
        space_optuna = {}
        for key,value in params_config['optuna'][model_name].items():
            space_optuna[key] = eval(value)
        model = model_class
        model.set_params(**space_optuna)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        cost = eval_metrics(y_true = y_test , y_pred = y_pred)
    
        return cost
tuner_report = {}
find_param=optuna.create_study(direction = "minimize")
find_param.optimize(optuna_objective,n_trials=1)

tuner_report['Optuna'] = {'cost':find_param.best_value, 'params': find_param.best_params}
print (f"Optuna: {model_name} --- {tuner_report['Optuna']}\n\n")

In [ ]:
find_param.best_params

In [ ]:
data = {'hidden_layer_sizes': (500, 300, 200, 150),
 'activation': 'tanh',
 'learning_rate': 'invscaling',
 'max_iter': 783}

In [ ]:
model.get_params()['grd_boost'].get_params()

In [ ]:
model.get_params()['log_reg']

In [ ]:
model.get_params()['xgb'].get_params()

In [ ]:
import ruamel.yaml as yaml
yaml_ = yaml.YAML()
with open('F:\iNeuron\Projects\scania_failures_2\sample.yaml', 'w') as f:
    yaml_.dump(stacked_classifer.get_params(), f)


In [ ]:
from src.utils import save_yaml


save_yaml(tuner_report,'F:\iNeuron\Projects\scania_failures_2\sample.yaml')

In [ ]:
file_path = r'F:\iNeuron\Projects\scania_failures_2\sample.json'

with open(file_path, 'w') as file:
    json.dump(obj=sc, fp=file, indent = 2)

In [ ]:
with open(file_path,'r') as file:
    data = json.load(file)
data

In [ ]:
load_yaml(Path('F:\iNeuron\Projects\scania_failures_2\sample.yaml'))

In [ ]:
import json
print(json.dumps(find_param.best_params))

In [ ]:
pwd

In [1]:
import yaml

with open(r"F:\iNeuron\Projects\scania_failures_2\mlflow_model_sources.yaml", "r") as f:
    model_names = yaml.safe_load(f)

model_names

{'Challenger_LGBMClassifier': 'mlflow-artifacts:/f2a1d0cb47994e43887a7bf8ff11fbc0/4f79ae8649044b24a593ec1c748e012b/artifacts/challenger_LGBMClassifier/model.pkl',
 'Challenger_Stacked_Classifier': 'mlflow-artifacts:/aacd27cc7a9d429ab2439e37a4e8cdfb/d3eb9ead8a5c49f9b8299efea62d58a2/artifacts/candidate_Stacked_Classifier/model.pkl',
 'Challenger_Voting_Classifier': 'mlflow-artifacts:/cd6eb5acbadd414fb5d32e02d983d99b/b2aea20ef6094e1a94b1567aea90753f/artifacts/candidate_Voting_Classifier/model.pkl'}

In [18]:
mlflow.search_registered_models(filter_string="tags.model ilike 'Light_GBM' AND tags.run_type ilike 'parent'")#[0].latest_versions[0].source

[]

In [52]:
exp_count = 282
mlflow.search_runs(experiment_names = [f"{exp_count}_Light_GBM_{exp_count}"],
                   filter_string = "tags.model ilike 'Light_GBM' and tags.run_type ilike 'parent'")['run_id'][0] #AND tags.run_type ilike 'parent")

'19190e88c698476496ef9f0f377da335'

In [40]:
mlflow.get_run('19190e88c698476496ef9f0f377da335').data.params


{'boosting_type': 'gbdt',
 'class_weight': 'balanced',
 'colsample_bytree': '1.0',
 'importance_type': 'split',
 'learning_rate': '0.5986318983049539',
 'max_depth': '-1',
 'min_child_samples': '20',
 'min_child_weight': '0.001',
 'min_split_gain': '0.0',
 'n_estimators': '145',
 'n_jobs': '-1',
 'num_leaves': '31',
 'objective': 'None',
 'random_state': 'None',
 'reg_alpha': '0.0',
 'reg_lambda': '0.0',
 'subsample': '1.0',
 'subsample_for_bin': '200000',
 'subsample_freq': '0'}

In [42]:
from src.utils import params_evaluator
params = params_evaluator(mlflow.get_run('19190e88c698476496ef9f0f377da335').data.params)

In [44]:
from lightgbm import LGBMClassifier
light_gbm = LGBMClassifier(**params)

In [46]:
bg_clf = BaggingClassifier(estimator=light_gbm)
bg_clf

BaggingClassifier(estimator=LGBMClassifier(class_weight='balanced',
                                           learning_rate=0.5986318983049539,
                                           n_estimators=145, n_jobs=-1))

In [47]:
bg_clf(**{'n_estimators':60})

TypeError: 'BaggingClassifier' object is not callable

In [4]:
live = Live()

In [10]:
live.__getstate__

<function Live.__getstate__()>

In [10]:
sources = {}
for i in range(2):
    model_name = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Challenger'")[i].name
    if model_name != 'Challenger Stacked_Classifier' and model_name != 'Challenger Voting_Classifier':
        model_name = 'Final_Estimator'
    else:
        model_name = model_name.replace(" ", "_")
    sources[model_name] = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Challenger'")[i].latest_versions[0].source + "/model.pkl"

In [11]:
for i in range(1):
    model_name = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[i].name
    if model_name != 'Challenger Stacked_Classifier' and model_name != 'Challenger Voting_Classifier':
        model_name = 'Final_Estimator'
    else:
        model_name = model_name.replace(" ", "_")
    sources[model_name] = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[i].latest_versions[0].source + "/model.pkl"

In [12]:
sources

{'Final_Estimator': 'mlflow-artifacts:/f2a1d0cb47994e43887a7bf8ff11fbc0/4f79ae8649044b24a593ec1c748e012b/artifacts/challenger_LGBMClassifier/model.pkl',
 'Challenger_Stacked_Classifier': 'mlflow-artifacts:/aacd27cc7a9d429ab2439e37a4e8cdfb/d3eb9ead8a5c49f9b8299efea62d58a2/artifacts/candidate_Stacked_Classifier/model.pkl',
 'Challenger_Voting_Classifier': 'mlflow-artifacts:/cd6eb5acbadd414fb5d32e02d983d99b/b2aea20ef6094e1a94b1567aea90753f/artifacts/candidate_Voting_Classifier/model.pkl'}

In [13]:
champion_source_ = {}
for i in range(1):
    model_name = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[i].name
    model_name = 'Champion_Estimator'
    champion_source_[model_name] = mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[i].latest_versions[0].source + "/model.pkl"

In [14]:
champion_source_

{'Champion_Estimator': 'mlflow-artifacts:/cd6eb5acbadd414fb5d32e02d983d99b/b2aea20ef6094e1a94b1567aea90753f/artifacts/candidate_Voting_Classifier/model.pkl'}

In [15]:
sources

{'Final_Estimator': 'mlflow-artifacts:/f2a1d0cb47994e43887a7bf8ff11fbc0/4f79ae8649044b24a593ec1c748e012b/artifacts/challenger_LGBMClassifier/model.pkl',
 'Challenger_Stacked_Classifier': 'mlflow-artifacts:/aacd27cc7a9d429ab2439e37a4e8cdfb/d3eb9ead8a5c49f9b8299efea62d58a2/artifacts/candidate_Stacked_Classifier/model.pkl',
 'Challenger_Voting_Classifier': 'mlflow-artifacts:/cd6eb5acbadd414fb5d32e02d983d99b/b2aea20ef6094e1a94b1567aea90753f/artifacts/candidate_Voting_Classifier/model.pkl'}

In [16]:
from src.utils import save_yaml

save_yaml(file = sources, filepath = r"F:\iNeuron\Projects\scania_failures_2\artifacts\model\model_sources.yaml", 
          mode = 'w')

[2024-02-08 10:29:07,489: INFO: utils: yaml file is saved]


In [17]:
from src.utils import save_yaml

save_yaml(file = champion_source_, filepath = r"F:\iNeuron\Projects\scania_failures_2\artifacts\model\model_sources.yaml", 
          mode = 'a')

[2024-02-08 10:29:34,416: INFO: utils: yaml file is saved]


In [5]:
import numpy as np


array([1.        , 1.11111111, 1.22222222, 1.33333333, 1.44444444,
       1.55555556, 1.66666667, 1.77777778, 1.88888889, 2.        ])

In [6]:
import pandas as pd

pd.DataFrame(np.linspace(1,2,10), columns = ['sample'])

,sample
0,1.000000
1,1.111111
2,1.222222
3,1.333333
4,1.444444
5,1.555556
6,1.666667
7,1.777778
8,1.888889
9,2.000000


In [4]:
mlflow.search_registered_models(filter_string="tags.model_type ilike 'Champion'")[0].latest_versions[0].source

'mlflow-artifacts:/60258e966e554f9880661b3db47d046e/2fefb228025247cca3c822e50b0484c8/artifacts/candidate_Voting_Classifier'

In [148]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\processed\stage_1_processing\preprocessed_train_data.csv")

In [44]:
y = df['class']

In [149]:
df['class'].value_counts()

class
0    59000
1     1000
Name: count, dtype: int64

In [38]:
df['class'].value_counts().mean()

30000.0

In [45]:
minority_class = y.value_counts().min()
majority_class = y.value_counts().max()
mean_count = int(y.value_counts().mean())
if minority_class <= mean_count-500 or majority_class >= mean_count+500:
    print("Dataset is imbalanced")

Dataset is imbalanced


In [41]:
mean_count-500

29500

In [43]:
if minority_class <= mean_count-500 or majority_class >= mean_count+500:
    print("Dataset is imbalanced")

Dataset is imbalanced


In [15]:
np.mean([29500,30500])

30000.0

In [16]:
29700 + 30300

60000

In [62]:
from datetime import datetime
timestamp = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
timestamp

'09_02_2024_12_24_44'

In [66]:
import pandas as pd
df = pd.DataFrame()
isinstance(df, pd.DataFrame)

True

In [67]:
from src.utils import load_yaml
from src.constants import SCHEMA_PATH

columns = load_yaml(SCHEMA_PATH).Features

[2024-02-09 16:42:11,023: INFO: utils: schema.yaml yaml_file is loaded]


In [86]:
columns_dict = load_yaml(SCHEMA_PATH).Features
columns = list(columns_dict.keys())
columns

[2024-02-09 16:45:21,292: INFO: utils: schema.yaml yaml_file is loaded]


['aa_000',
 'ab_000',
 'ac_000',
 'ad_000',
 'ae_000',
 'af_000',
 'ag_000',
 'ag_001',
 'ag_002',
 'ag_003',
 'ag_004',
 'ag_005',
 'ag_006',
 'ag_007',
 'ag_008',
 'ag_009',
 'ah_000',
 'ai_000',
 'aj_000',
 'ak_000',
 'al_000',
 'am_0',
 'an_000',
 'ao_000',
 'ap_000',
 'aq_000',
 'ar_000',
 'as_000',
 'at_000',
 'au_000',
 'av_000',
 'ax_000',
 'ay_000',
 'ay_001',
 'ay_002',
 'ay_003',
 'ay_004',
 'ay_005',
 'ay_006',
 'ay_007',
 'ay_008',
 'ay_009',
 'az_000',
 'az_001',
 'az_002',
 'az_003',
 'az_004',
 'az_005',
 'az_006',
 'az_007',
 'az_008',
 'az_009',
 'ba_000',
 'ba_001',
 'ba_002',
 'ba_003',
 'ba_004',
 'ba_005',
 'ba_006',
 'ba_007',
 'ba_008',
 'ba_009',
 'bb_000',
 'bc_000',
 'bd_000',
 'be_000',
 'bf_000',
 'bg_000',
 'bh_000',
 'bi_000',
 'bj_000',
 'bk_000',
 'bl_000',
 'bm_000',
 'bn_000',
 'bo_000',
 'bp_000',
 'bq_000',
 'br_000',
 'bs_000',
 'bt_000',
 'bu_000',
 'bv_000',
 'bx_000',
 'by_000',
 'bz_000',
 'ca_000',
 'cb_000',
 'cc_000',
 'cd_000',
 'ce_000',
 

In [142]:
len(columns)

170

In [83]:
pprint(list(columns.keys()))

Pretty printing has been turned ON


In [118]:
np.array(list(df.iloc[:1,:].to_dict(orient='records')[0].values())).shape

(171,)

In [120]:
np.array(list(df.iloc[:1,:].to_dict(orient='records')[0].values())).reshape(1, -1).shape

(1, 171)

In [124]:
isinstance(df.iloc[:1,:].to_dict(orient='records'),list)

True

In [127]:
df.iloc[:1,:].to_dict(orient='records')

[{'aa_000': 76698.0,
  'ab_000': nan,
  'ac_000': 2130706438.0,
  'ad_000': 280.0,
  'ae_000': 0.0,
  'af_000': 0.0,
  'ag_000': 0.0,
  'ag_001': 0.0,
  'ag_002': 0.0,
  'ag_003': 0.0,
  'ag_004': 37250.0,
  'ag_005': 1432864.0,
  'ag_006': 3664156.0,
  'ag_007': 1007684.0,
  'ag_008': 25896.0,
  'ag_009': 0.0,
  'ah_000': 2551696.0,
  'ai_000': 0.0,
  'aj_000': 0.0,
  'ak_000': 0.0,
  'al_000': 0.0,
  'am_0': 0.0,
  'an_000': 4933296.0,
  'ao_000': 3655166.0,
  'ap_000': 1766008.0,
  'aq_000': 1132040.0,
  'ar_000': 0.0,
  'as_000': 0.0,
  'at_000': 0.0,
  'au_000': 0.0,
  'av_000': 1012.0,
  'ax_000': 268.0,
  'ay_000': 0.0,
  'ay_001': 0.0,
  'ay_002': 0.0,
  'ay_003': 0.0,
  'ay_004': 0.0,
  'ay_005': 469014.0,
  'ay_006': 4239660.0,
  'ay_007': 703300.0,
  'ay_008': 755876.0,
  'ay_009': 0.0,
  'az_000': 5374.0,
  'az_001': 2108.0,
  'az_002': 4114.0,
  'az_003': 12348.0,
  'az_004': 615248.0,
  'az_005': 5526276.0,
  'az_006': 2378.0,
  'az_007': 4.0,
  'az_008': 0.0,
  'az_009':

In [135]:
from src.utils import load_binary


preprocessor = load_binary(r"artifacts/preprocessor/preprocessor.joblib")

[2024-02-09 17:31:53,603: INFO: utils: pickled_object: artifacts/preprocessor/preprocessor.joblib loaded]


In [146]:
preprocessor.transform(np.linspace(1,50,170).reshape(1,-1))

array([[-6.47730021e-01,  1.11242604e+00, -1.17422791e-01,
        -4.06846209e-01,  2.15976331e+00,  2.44970414e+00,
         2.73964497e+00,  3.02958580e+00,  3.31952663e+00,
         3.60946746e+00, -7.56968324e-02, -1.95480411e-01,
        -4.98467929e-01, -2.02903234e-01, -6.70214036e-02,
         1.37156729e-02, -6.38748503e-01,  5.92899408e+00,
         6.21893491e+00,  6.50887574e+00,  5.40446468e-03,
         2.93651922e-03, -6.27698237e-01, -6.30878640e-01,
        -5.12166775e-01, -4.80770141e-01,  8.53846154e+00,
         8.82840237e+00,  9.11834320e+00,  9.40828402e+00,
        -1.73973199e-01, -2.26320563e-01,  1.02781065e+01,
         1.05680473e+01,  1.08579882e+01,  1.11479290e+01,
         1.14378698e+01,  2.87516809e-04, -1.33809563e-01,
        -2.63182106e-01, -1.56714121e-01,  1.28875740e+01,
        -6.65013565e-01, -3.18083271e-01, -3.29060902e-01,
        -8.33466557e-02, -4.69995918e-02, -3.00188511e-01,
        -6.83205236e-02,  1.52071006e+01,  1.54970414e+0

In [141]:
np.linspace(1,50,170).reshape(1,-1)

(1, 170)

In [134]:
np.ndarray(list(np.linspace(1,50,1)))

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [4]:
from pprint import pprint
import pandas as pd


pprint(pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\train_test\train.csv").columns.tolist(),compact = True)

['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000', 'ag_001',
 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009',
 'ah_000', 'ai_000', 'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000',
 'ap_000', 'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000',
 'ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007',
 'ay_008', 'ay_009', 'az_000', 'az_001', 'az_002', 'az_003', 'az_004', 'az_005',
 'az_006', 'az_007', 'az_008', 'az_009', 'ba_000', 'ba_001', 'ba_002', 'ba_003',
 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009', 'bb_000', 'bc_000',
 'bd_000', 'be_000', 'bf_000', 'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000',
 'bl_000', 'class', 'bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000',
 'bs_000', 'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000',
 'cb_000', 'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000',
 'cj_000', 'ck_000', 'cl_000', 

In [5]:
len(['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000', 'ag_001',
 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009',
 'ah_000', 'ai_000', 'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000',
 'ap_000', 'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000',
 'ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007',
 'ay_008', 'ay_009', 'az_000', 'az_001', 'az_002', 'az_003', 'az_004', 'az_005',
 'az_006', 'az_007', 'az_008', 'az_009'])

52

In [24]:
len(['ba_000', 'ba_001', 'ba_002', 'ba_003',
 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009', 'bb_000', 'bc_000',
 'bd_000', 'be_000', 'bf_000', 'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000',
 'bl_000', 'class', 'bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000',
 'bs_000', 'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000'])

35

In [7]:
len(['ca_000',
 'cb_000', 'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000',
 'cj_000', 'ck_000', 'cl_000', 'cm_000', 'cn_000', 'cn_001', 'cn_002', 'cn_003',
 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008', 'cn_009', 'co_000', 'cp_000',
 'cq_000', 'cr_000', 'cs_000', 'cs_001', 'cs_002', 'cs_003', 'cs_004', 'cs_005',
 'cs_006', 'cs_007', 'cs_008', 'cs_009', 'ct_000', 'cu_000', 'cv_000', 'cx_000',
 'cy_000', 'cz_000'])

43

In [8]:
len(['da_000', 'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000',
 'dg_000', 'dh_000', 'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000',
 'do_000', 'dp_000', 'dq_000', 'dr_000', 'ds_000', 'dt_000', 'du_000', 'dv_000',
 'dx_000', 'dy_000', 'dz_000'])

25

In [9]:
len(['ea_000', 'eb_000', 'ec_00', 'ed_000', 'ee_000',
 'ee_001', 'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
 'ee_009', 'ef_000', 'eg_000'])

16

In [3]:
series = ['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000',
'af_000', 'ag_000', 'ag_001', 'ag_002', 'ag_003', 'ag_004',
'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009', 'ah_000',
'ai_000', 'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000',
'ao_000', 'ap_000', 'aq_000', 'ar_000', 'as_000', 'at_000',
'au_000', 'av_000', 'ax_000', 'ay_000', 'ay_001', 'ay_002',
'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007', 'ay_008',
'ay_009', 'az_000', 'az_001', 'az_002', 'az_003', 'az_004',
'az_005', 'az_006', 'az_007', 'az_008', 'az_009','ba_000', 'ba_001', 'ba_002', 'ba_003',
 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009', 'bb_000', 'bc_000',
 'bd_000', 'be_000', 'bf_000', 'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000',
 'bl_000', 'class', 'bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000',
 'bs_000', 'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000']

In [21]:
a_series = [s for s in series if s.startswith('a')]
b_series = [s for s in series if s.startswith('b')]


In [33]:
from src.utils import load_yaml
from src.constants import SCHEMA_PATH
columns_dict = load_yaml(SCHEMA_PATH).Features
a_series = [s for s in columns_dict if s.startswith('a')]
b_series = [s for s in columns_dict if s.startswith('b')]
c_series = [s for s in columns_dict if s.startswith('c')]
d_series = [s for s in columns_dict if s.startswith('d')]
e_series = [s for s in columns_dict if s.startswith('e')]

[2024-02-11 20:30:14,460: INFO: utils: schema.yaml yaml_file is loaded]


In [28]:
b_series

['ba_000',
 'ba_001',
 'ba_002',
 'ba_003',
 'ba_004',
 'ba_005',
 'ba_006',
 'ba_007',
 'ba_008',
 'ba_009',
 'bb_000',
 'bc_000',
 'bd_000',
 'be_000',
 'bf_000',
 'bg_000',
 'bh_000',
 'bi_000',
 'bj_000',
 'bk_000',
 'bl_000',
 'bm_000',
 'bn_000',
 'bo_000',
 'bp_000',
 'bq_000',
 'br_000',
 'bs_000',
 'bt_000',
 'bu_000',
 'bv_000',
 'bx_000',
 'by_000',
 'bz_000']

In [29]:
c_series

['ca_000',
 'cb_000',
 'cc_000',
 'cd_000',
 'ce_000',
 'cf_000',
 'cg_000',
 'ch_000',
 'ci_000',
 'cj_000',
 'ck_000',
 'cl_000',
 'cm_000',
 'cn_000',
 'cn_001',
 'cn_002',
 'cn_003',
 'cn_004',
 'cn_005',
 'cn_006',
 'cn_007',
 'cn_008',
 'cn_009',
 'co_000',
 'cp_000',
 'cq_000',
 'cr_000',
 'cs_000',
 'cs_001',
 'cs_002',
 'cs_003',
 'cs_004',
 'cs_005',
 'cs_006',
 'cs_007',
 'cs_008',
 'cs_009',
 'ct_000',
 'cu_000',
 'cv_000',
 'cx_000',
 'cy_000',
 'cz_000']

In [30]:
e_series

['ea_000',
 'eb_000',
 'ec_00',
 'ed_000',
 'ee_000',
 'ee_001',
 'ee_002',
 'ee_003',
 'ee_004',
 'ee_005',
 'ee_006',
 'ee_007',
 'ee_008',
 'ee_009',
 'ef_000',
 'eg_000']

In [32]:
d_series

['da_000',
 'db_000',
 'dc_000',
 'dd_000',
 'de_000',
 'df_000',
 'dg_000',
 'dh_000',
 'di_000',
 'dj_000',
 'dk_000',
 'dl_000',
 'dm_000',
 'dn_000',
 'do_000',
 'dp_000',
 'dq_000',
 'dr_000',
 'ds_000',
 'dt_000',
 'du_000',
 'dv_000',
 'dx_000',
 'dy_000',
 'dz_000']

In [34]:
a_series

['aa_000',
 'ab_000',
 'ac_000',
 'ad_000',
 'ae_000',
 'af_000',
 'ag_000',
 'ag_001',
 'ag_002',
 'ag_003',
 'ag_004',
 'ag_005',
 'ag_006',
 'ag_007',
 'ag_008',
 'ag_009',
 'ah_000',
 'ai_000',
 'aj_000',
 'ak_000',
 'al_000',
 'am_0',
 'an_000',
 'ao_000',
 'ap_000',
 'aq_000',
 'ar_000',
 'as_000',
 'at_000',
 'au_000',
 'av_000',
 'ax_000',
 'ay_000',
 'ay_001',
 'ay_002',
 'ay_003',
 'ay_004',
 'ay_005',
 'ay_006',
 'ay_007',
 'ay_008',
 'ay_009',
 'az_000',
 'az_001',
 'az_002',
 'az_003',
 'az_004',
 'az_005',
 'az_006',
 'az_007',
 'az_008',
 'az_009']

In [1]:
import pandas as pd
df = pd.read_csv(r"F:\iNeuron\Projects\Data\Scania Truck Failures\aps_failure_test_set.csv")

In [3]:
y=df['class']

In [15]:
'neg' in list(y) or 'pos' in list(y)

True

In [29]:
flag = 0
for i in df.columns:
    if 'na' in list(df[i]):
        flag =1

In [8]:
data = {'aa_000': '1800546',
 'ab_000': 'na',
 'ac_000': 'na',
 'ad_000': '13042',
 'ae_000': 'na',
 'af_000': 'na',
 'ag_000': '0',
 'ag_001': '280',
 'ag_002': '132164',
 'ag_003': '7875494',
 'ag_004': '55386332',
 'ag_005': '92749198',
 'ag_006': '52130878',
 'ag_007': '16496844',
 'ag_008': '1901366',
 'ag_009': '28480',
 'ah_000': '82073576',
 'ai_000': '196142',
 'aj_000': '0',
 'ak_000': 'na',
 'al_000': '986196',
 'am_0': '1957344',
 'an_000': '149868570',
 'ao_000': '115528598',
 'ap_000': '84129814',
 'aq_000': '34558772',
 'ar_000': 'na',
 'as_000': '0',
 'at_000': '0',
 'au_000': '0',
 'av_000': 'na',
 'ax_000': 'na',
 'ay_000': '0',
 'ay_001': '0',
 'ay_002': '0',
 'ay_003': '0',
 'ay_004': '0',
 'ay_005': '0',
 'ay_006': '0',
 'ay_007': '28112774',
 'ay_008': '198588262',
 'ay_009': '0',
 'az_000': '119430',
 'az_001': '77174',
 'az_002': '121294',
 'az_003': '274336',
 'az_004': '81066740',
 'az_005': '145041998',
 'az_006': '64',
 'az_007': '0',
 'az_008': '0',
 'az_009': '0'}

In [9]:
for key,value in data.items():
    try:
        data[key] = eval(value)
    except Exception:
        pass

pd.DataFrame(data,index = [0])

In [10]:
data

{'aa_000': 1800546,
 'ab_000': 'na',
 'ac_000': 'na',
 'ad_000': 13042,
 'ae_000': 'na',
 'af_000': 'na',
 'ag_000': 0,
 'ag_001': 280,
 'ag_002': 132164,
 'ag_003': 7875494,
 'ag_004': 55386332,
 'ag_005': 92749198,
 'ag_006': 52130878,
 'ag_007': 16496844,
 'ag_008': 1901366,
 'ag_009': 28480,
 'ah_000': 82073576,
 'ai_000': 196142,
 'aj_000': 0,
 'ak_000': 'na',
 'al_000': 986196,
 'am_0': 1957344,
 'an_000': 149868570,
 'ao_000': 115528598,
 'ap_000': 84129814,
 'aq_000': 34558772,
 'ar_000': 'na',
 'as_000': 0,
 'at_000': 0,
 'au_000': 0,
 'av_000': 'na',
 'ax_000': 'na',
 'ay_000': 0,
 'ay_001': 0,
 'ay_002': 0,
 'ay_003': 0,
 'ay_004': 0,
 'ay_005': 0,
 'ay_006': 0,
 'ay_007': 28112774,
 'ay_008': 198588262,
 'ay_009': 0,
 'az_000': 119430,
 'az_001': 77174,
 'az_002': 121294,
 'az_003': 274336,
 'az_004': 81066740,
 'az_005': 145041998,
 'az_006': 64,
 'az_007': 0,
 'az_008': 0,
 'az_009': 0}

In [12]:
import pandas as pd
pd.DataFrame(data,index = [0])

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,az_000,az_001,az_002,az_003,az_004,az_005,az_006,az_007,az_008,az_009
0,1800546,na,na,13042,na,na,0,280,132164,7875494,...,119430,77174,121294,274336,81066740,145041998,64,0,0,0


In [9]:
preprocessor = load_binary(r"artifacts\preprocessor\preprocessor.joblib")
preprocessor

[2024-02-13 07:41:38,543: INFO: utils: pickled_object: artifacts\preprocessor\preprocessor.joblib loaded]


Pipeline(steps=[('Knn_imputer', KNNImputer()),
                ('Robust_Scaler', RobustScaler())],
         verbose=True)

In [12]:
list_from_processor = preprocessor.get_feature_names_out()
list(list_from_processor)

['aa_000',
 'ab_000',
 'ac_000',
 'ad_000',
 'ae_000',
 'af_000',
 'ag_000',
 'ag_001',
 'ag_002',
 'ag_003',
 'ag_004',
 'ag_005',
 'ag_006',
 'ag_007',
 'ag_008',
 'ag_009',
 'ah_000',
 'ai_000',
 'aj_000',
 'ak_000',
 'al_000',
 'am_0',
 'an_000',
 'ao_000',
 'ap_000',
 'aq_000',
 'ar_000',
 'as_000',
 'at_000',
 'au_000',
 'av_000',
 'ax_000',
 'ay_000',
 'ay_001',
 'ay_002',
 'ay_003',
 'ay_004',
 'ay_005',
 'ay_006',
 'ay_007',
 'ay_008',
 'ay_009',
 'az_000',
 'az_001',
 'az_002',
 'az_003',
 'az_004',
 'az_005',
 'az_006',
 'az_007',
 'az_008',
 'az_009',
 'ba_000',
 'ba_001',
 'ba_002',
 'ba_003',
 'ba_004',
 'ba_005',
 'ba_006',
 'ba_007',
 'ba_008',
 'ba_009',
 'bb_000',
 'bc_000',
 'bd_000',
 'be_000',
 'bf_000',
 'bg_000',
 'bh_000',
 'bi_000',
 'bj_000',
 'bk_000',
 'bl_000',
 'bm_000',
 'bn_000',
 'bo_000',
 'bp_000',
 'bq_000',
 'br_000',
 'bs_000',
 'bt_000',
 'bu_000',
 'bv_000',
 'bx_000',
 'by_000',
 'bz_000',
 'ca_000',
 'cb_000',
 'cc_000',
 'cd_000',
 'ce_000',
 

In [4]:
data_from_flask = {'aa_000': '1',
 'ab_000': 'na',
 'ac_000': '1',
 'ad_000': '1',
 'ae_000': '1',
 'af_000': '1',
 'ag_000': '1',
 'ag_001': '1',
 'ag_002': '1',
 'ag_003': '1',
 'ag_004': '1',
 'ag_005': '1',
 'ag_006': '1',
 'ag_007': '1',
 'ag_008': '1',
 'ag_009': '1',
 'ah_000': '1',
 'ai_000': '1',
 'aj_000': '1',
 'ak_000': '1',
 'al_000': '1',
 'am_0': '1',
 'an_000': '1',
 'ao_000': '11',
 'ap_000': '1',
 'aq_000': '1',
 'ar_000': '1',
 'as_000': '1',
 'at_000': '1',
 'au_000': '1',
 'av_000': '1',
 'ax_000': '1',
 'ay_000': '1',
 'ay_001': '1',
 'ay_002': '1',
 'ay_003': '1',
 'ay_004': '1',
 'ay_005': '1',
 'ay_006': '1',
 'ay_007': '1',
 'ay_008': '1',
 'ay_009': '1',
 'az_000': '1',
 'az_001': '1',
 'az_002': '1',
 'az_003': '1',
 'az_004': '1',
 'az_005': '1',
 'az_006': '1',
 'az_007': '1',
 'az_008': '1',
 'az_009': '1',
 'ba_000': '1',
 'ba_001': '1',
 'ba_002': '1',
 'ba_003': '1',
 'ba_004': '1',
 'ba_005': 'na',
 'ba_006': 'na',
 'ba_007': 'na',
 'ba_008': '11',
 'ba_009': '1',
 'bb_000': '1',
 'bc_000': '1',
 'bd_000': '1',
 'be_000': '1',
 'bf_000': '1',
 'bg_000': '1',
 'bh_000': '1',
 'bi_000': '1',
 'bj_000': '1',
 'bk_000': '1',
 'bl_000': '1',
 'bm_000': '1',
 'bn_000': '1',
 'bo_000': '1',
 'bp_000': '1',
 'bq_000': '1',
 'br_000': '1',
 'bs_000': '1',
 'bt_000': '1',
 'bu_000': '1',
 'bv_000': '1',
 'bx_000': '1',
 'by_000': '1',
 'bz_000': '1',
 'ca_000': '2',
 'cb_000': '4',
 'cc_000': '5',
 'cd_000': '3',
 'ce_000': '4',
 'cf_000': 'na',
 'cg_000': 'na',
 'ch_000': '1',
 'ci_000': '1',
 'cj_000': '1',
 'ck_000': '1',
 'cl_000': '1',
 'cm_000': '1',
 'cn_000': '1',
 'cn_001': '1',
 'cn_002': '1',
 'cn_003': '1',
 'cn_004': '1',
 'cn_005': '1',
 'cn_006': '1',
 'cn_007': '1',
 'cn_008': '1',
 'cn_009': '1',
 'co_000': '1',
 'cp_000': '1',
 'cq_000': '1',
 'cr_000': '1',
 'cs_000': '1',
 'cs_001': '1',
 'cs_002': '1',
 'cs_003': '1',
 'cs_004': '1',
 'cs_005': '1',
 'cs_006': '1',
 'cs_007': '1',
 'cs_008': '1',
 'cs_009': '1',
 'ct_000': '1',
 'cu_000': '1',
 'cv_000': '1',
 'cx_000': '1',
 'cy_000': '1',
 'cz_000': '1',
 'da_000': '2',
 'db_000': '5',
 'dc_000': '8',
 'dd_000': '4',
 'de_000': '13',
 'df_000': '8',
 'dg_000': '14',
 'dh_000': '8',
 'di_000': '4',
 'dj_000': '89',
 'dk_000': '4',
 'dl_000': '54',
 'dm_000': '8',
 'dn_000': '4',
 'do_000': '8',
 'dp_000': '14',
 'dq_000': '98',
 'dr_000': '1',
 'ds_000': '4',
 'dt_000': '9',
 'du_000': '7',
 'dv_000': '5',
 'dx_000': '6',
 'dy_000': '74',
 'dz_000': '4',
 'ea_000': '9',
 'eb_000': '8',
 'ec_00': '7',
 'ed_000': '6',
 'ee_000': '2',
 'ee_001': '5',
 'ee_002': '4',
 'ee_003': '1',
 'ee_004': '3',
 'ee_005': '5',
 'ee_006': '96',
 'ee_007': '8',
 'ee_008': '7',
 'ee_009': '12',
 'ef_000': '5',
 'eg_000': '9'}

In [13]:
flag = 0 
for i,key in enumerate(data_from_flask.keys()):
    print("DICT_KEY: ",key)
    print("i: ",i)
    if list_from_processor[i] == key:
        flag = 0
    else:
        flag = 1
    print("LIST KEY: ",list_from_processor[i])

DICT_KEY:  aa_000
i:  0
LIST KEY:  aa_000
DICT_KEY:  ab_000
i:  1
LIST KEY:  ab_000
DICT_KEY:  ac_000
i:  2
LIST KEY:  ac_000
DICT_KEY:  ad_000
i:  3
LIST KEY:  ad_000
DICT_KEY:  ae_000
i:  4
LIST KEY:  ae_000
DICT_KEY:  af_000
i:  5
LIST KEY:  af_000
DICT_KEY:  ag_000
i:  6
LIST KEY:  ag_000
DICT_KEY:  ag_001
i:  7
LIST KEY:  ag_001
DICT_KEY:  ag_002
i:  8
LIST KEY:  ag_002
DICT_KEY:  ag_003
i:  9
LIST KEY:  ag_003
DICT_KEY:  ag_004
i:  10
LIST KEY:  ag_004
DICT_KEY:  ag_005
i:  11
LIST KEY:  ag_005
DICT_KEY:  ag_006
i:  12
LIST KEY:  ag_006
DICT_KEY:  ag_007
i:  13
LIST KEY:  ag_007
DICT_KEY:  ag_008
i:  14
LIST KEY:  ag_008
DICT_KEY:  ag_009
i:  15
LIST KEY:  ag_009
DICT_KEY:  ah_000
i:  16
LIST KEY:  ah_000
DICT_KEY:  ai_000
i:  17
LIST KEY:  ai_000
DICT_KEY:  aj_000
i:  18
LIST KEY:  aj_000
DICT_KEY:  ak_000
i:  19
LIST KEY:  ak_000
DICT_KEY:  al_000
i:  20
LIST KEY:  al_000
DICT_KEY:  am_0
i:  21
LIST KEY:  am_0
DICT_KEY:  an_000
i:  22
LIST KEY:  an_000
DICT_KEY:  ao_000
i:  23


In [18]:
list_from_processor[0]

'aa_000'

In [17]:
pd.unique(list(data_from_flask.keys()) == list_from_processor)

array([ True])